In [1]:
import sqlite3
import pandas as pd
import glob


In [2]:
con = sqlite3.connect(r'C:\Users\sasha\Documents\Мои документы\2021\Grisha\Cells.db')
cursorObj = con.cursor()

In [3]:
cursorObj.execute('''CREATE TABLE intensity_vals (
    id                     INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Mean_GFP_intensity     REAL,
    Mean_Tubulin_intensity REAL,
    Std_GFP_intensity      REAL,
    Std_Tubulin_intensity  REAL,
    phenotype              TEXT    REFERENCES phenotype (id) ON DELETE RESTRICT
                                                             ON UPDATE CASCADE
);''')
con.commit()

In [4]:
path = r"C:\Users\sasha\Documents\Мои документы\2021\Grisha"
all_files = glob.glob(path + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['phenotype'] = filename[51:53]
    li.append(df)

all_data = pd.concat(li, axis=0, ignore_index=True)

In [5]:
data  = all_data[['Intensity_MeanIntensity_GFP','Intensity_MeanIntensity_Tubulin','Intensity_StdIntensity_GFP','Intensity_StdIntensity_Tubulin', 'phenotype']]
data.head()
data = data.rename(columns = {'Intensity_MeanIntensity_GFP' : 'Mean_GFP_intensity',
                      'Intensity_MeanIntensity_Tubulin': 'Mean_Tubulin_intensity',
                      'Intensity_StdIntensity_GFP' : 'Std_GFP_intensity',
                      'Intensity_StdIntensity_Tubulin' : 'Std_Tubulin_intensity',
                      'phenotype' : 'phenotype'})
data.replace(to_replace={'Em': 1, 'Wt': 2, 'Mu': 3}, inplace=True)
data.head()

,Mean_GFP_intensity,Mean_Tubulin_intensity,Std_GFP_intensity,Std_Tubulin_intensity,phenotype
0,0.018797,0.136437,0.001900,0.044344,1
1,0.018785,0.058284,0.001803,0.016634,1
2,0.023652,0.178897,0.005090,0.074361,1
3,0.019133,0.116139,0.001885,0.025173,1
4,0.019407,0.153829,0.001974,0.051390,1


In [6]:
data.to_sql(name = 'intensity_vals', con= con, if_exists = 'append',  index_label='id')


In [7]:
con.commit()

In [8]:
cursorObj.execute('''CREATE TABLE phenotype (
    id        INTEGER PRIMARY KEY AUTOINCREMENT
                      NOT NULL,
    phenotype TEXT    UNIQUE
                      NOT NULL,
    mutation  TEXT    UNIQUE
                      NOT NULL
)''')

con.commit()
records = [('Emty', 'GFP plasmid'),
           ('Wild type', 'TBCE GFP'),
           ('Mutant', 'TBCE 112W-A GFP')]
cursorObj.executemany('INSERT INTO phenotype (phenotype, mutation) VALUES(?,?);',records);
con.commit()
cursorObj.execute('PRAGMA foreign_keys=on;')
con.close()